# Немного кода для обкачивания интернета
## Модуль [requests](https://realpython.com/python-requests/)

Давайте посмотрим html формат

Лучший учебник - https://www.w3schools.com

![](https://raw.githubusercontent.com/tbkazakova/compling_for_lyceum/main/data/tag.png "tag")

Вот пример очень простого сайта в интернете https://itcorp.com/, зарегистрированного в 1986 году.

И сайта посложнее https://spork.org/, посвящённый ложковилке (последнее обновление в 1996).

Теперь скачаем html страничку

In [1]:
import requests

In [2]:
URL = 'https://zoonovosib.ru/news/skolko-malyshey-rozhdaetsya-v-novosibirskom-zooparke-ezhegodno/'
response = requests.get(URL, verify=False)

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zoonovosib.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [6]:
# отключим предупреждения
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings
disable_warnings(InsecureRequestWarning)

In [7]:
response = requests.get(URL, verify=False)

В `response` теперь лежит ответ сервера. Это не просто html-код страницы, а еще дополнительная информация

In [8]:
response

<Response [200]>

In [6]:
response.status_code
# 200 - "всё работает"
# 404, 503 и др. - "произошла ошибка, не работает, такой страницы нет и т.д."

200

In [9]:
# Вывели первые 210 символов html
print(response.text[:210])

<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">

    <meta name="viewport" content="width=device-width">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">

    <link rel="shortcut icon" type="i


### Что делать, если сайт защищается?

Если он защищается от краулеров, можно, например, представиться Мозиллой:

In [8]:
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'  # хотим притворяться браузером

response = requests.get(URL, headers={'User-Agent':user_agent}, verify=False)

Или использовать **специальную библиотеку**:

In [27]:
! pip3 install fake-useragent

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 3.3 MB/s eta 0:00:00


In [28]:
from fake_useragent import UserAgent

In [29]:
user_agent = UserAgent().chrome
user_agent

'Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Mobile Safari/537.36'

In [13]:
response = requests.get(URL,
                        headers={'User-Agent':user_agent},
                        verify=False)

Или **прокси-сервер**. Это дополнительное звено между вами и интернетом, через него пойдет подключение и сайт не будет знать, что это вы посылаете запрос.

Адреса прокси можно взять со специальных сайтов. И потом проверить, что они рабочие, прежде чем использовать https://checkerproxy.net/

[Пример сервиса](https://whatismyipaddress.com/ip-lookup), который позволяет с некоторой точностью отследить по IP адресу

Возвращаемся к скачиванию странички.

In [10]:
response

<Response [200]>

In [11]:
response.url  #ссылка

'https://zoonovosib.ru/news/skolko-malyshey-rozhdaetsya-v-novosibirskom-zooparke-ezhegodno/'

In [12]:
print(response.text[:300])  #html код

<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">

    <meta name="viewport" content="width=device-width">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">

    <link rel="shortcut icon" type="image/x-icon" href="/favicon.ico" />
    <title>СКОЛЬКО МАЛЫШЕЙ РОЖДАЕТСЯ В НОВОСИБИРСКОМ З


Существует несколько вариантов, как достать что-то из определенного тега, например, достать заголовок:

- регулярные выражения (плохой вариант)
- специальные библиотеки питона, например, BeautifulSoup (bs4) или lxml (хороший вариант)


Регулярными выражениями мы уже умеем:

In [13]:
import re

In [14]:
code = response.text
re.search(r'<div class="detail-text">(.|\n)*?<\/div>', code).group()

'<div class="detail-text">\n            Новосибирский зоопарк регулярно рассказывает о детенышах, появившихся на свет в семьях животных. Ежегодно мы публикуем десятки историй о малышах. При этом многое остается нерассказанным, потому что коллекция очень большая. Но все события, все рождения фиксируются в документах, остаются в нашей летописи. Сегодня мы публикуем некоторые цифры, которые показывают результаты работы зоологического парка по получению потомства от животных и сохранению биологического разнообразия. <br>\r\n <br>\r\n В настоящее время в коллекции Новосибирского зоопарка представлено 803 вида, свыше 10 тысяч особей. В 2024 году было получено потомство от более 270 видов животных. Малыши рождались в семьях хищников, копытных, птиц, приматов, грызунов, рептилий, и все получали максимум заботы. <br>\r\n <br>\r\n У млекопитающих в общей сложности родилось более 250 детенышей. Среди них были малыши редких исчезающих видов, и каждый такой детеныш очень важен. Историческим событие

Теперь умный способ:

In [15]:
from bs4 import BeautifulSoup

In [16]:
# инициализируем (создаем) soup из response.text (кода страницы)
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.prettify()[:500])  # печатаем приукрашенный суп

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width" name="viewport"/>
  <meta content="ie=edge" http-equiv="X-UA-Compatible"/>
  <link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <title>
   СКОЛЬКО МАЛЫШЕЙ РОЖДАЕТСЯ В НОВОСИБИРСКОМ ЗООПАРКЕ ЕЖЕГОДНО
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type">
   <meta content="index, follow" name="robots">
    <meta content="новости новосибирского зоопарка, зоопарк


Если мы хотим вывести заголовок, нужно посмотреть, какими тегами он оформляется.

Заходим на наш сайт

На этом конкретном сайте заголовки лежат внутри тегов `cp-banner`

In [22]:
name = soup.find('h1')
print(name.prettify())

<h1>
 СКОЛЬКО МАЛЫШЕЙ РОЖДАЕТСЯ В НОВОСИБИРСКОМ ЗООПАРКЕ ЕЖЕГОДНО
</h1>



In [18]:
name

<h1>СКОЛЬКО МАЛЫШЕЙ РОЖДАЕТСЯ В НОВОСИБИРСКОМ ЗООПАРКЕ ЕЖЕГОДНО</h1>

Нас интересует только текст, поэтому сделаем `get_text`

In [19]:
print(name.get_text())

СКОЛЬКО МАЛЫШЕЙ РОЖДАЕТСЯ В НОВОСИБИРСКОМ ЗООПАРКЕ ЕЖЕГОДНО


Если бы на странице было несколько  `<h1>`, нужно было бы подробнее прописать, где лежит именно наш тег.

Например, наш тег лежит в теге `<main>`, поэтому мы могли сделать `soup.select('main' > 'h1')`, это бы выдало нам все `<h1>` внутри `<main>`.

А теперь найдём текст новости.

Он лежит в теге `<div class="detail-text">`

\- Как устроен этот тег?

\- div - имя тега, class - имя атрибута, "detail-text" - значение атрибута (атрибутов может быть много)

In [23]:
post = soup.find('div', {'class':"detail-text"})  # вот так можно указать атрибуты тегов
print(post.prettify())

<div class="detail-text">
 Новосибирский зоопарк регулярно рассказывает о детенышах, появившихся на свет в семьях животных. Ежегодно мы публикуем десятки историй о малышах. При этом многое остается нерассказанным, потому что коллекция очень большая. Но все события, все рождения фиксируются в документах, остаются в нашей летописи. Сегодня мы публикуем некоторые цифры, которые показывают результаты работы зоологического парка по получению потомства от животных и сохранению биологического разнообразия.
 <br/>
 <br/>
 В настоящее время в коллекции Новосибирского зоопарка представлено 803 вида, свыше 10 тысяч особей. В 2024 году было получено потомство от более 270 видов животных. Малыши рождались в семьях хищников, копытных, птиц, приматов, грызунов, рептилий, и все получали максимум заботы.
 <br/>
 <br/>
 У млекопитающих в общей сложности родилось более 250 детенышей. Среди них были малыши редких исчезающих видов, и каждый такой детеныш очень важен. Историческим событием стало рождение да

In [24]:
text = post.get_text()  # получили текст
print(text)


            Новосибирский зоопарк регулярно рассказывает о детенышах, появившихся на свет в семьях животных. Ежегодно мы публикуем десятки историй о малышах. При этом многое остается нерассказанным, потому что коллекция очень большая. Но все события, все рождения фиксируются в документах, остаются в нашей летописи. Сегодня мы публикуем некоторые цифры, которые показывают результаты работы зоологического парка по получению потомства от животных и сохранению биологического разнообразия. 

 В настоящее время в коллекции Новосибирского зоопарка представлено 803 вида, свыше 10 тысяч особей. В 2024 году было получено потомство от более 270 видов животных. Малыши рождались в семьях хищников, копытных, птиц, приматов, грызунов, рептилий, и все получали максимум заботы. 

 У млекопитающих в общей сложности родилось более 250 детенышей. Среди них были малыши редких исчезающих видов, и каждый такой детеныш очень важен. Историческим событием стало рождение дальневосточного леопарда. Среди знаковы

In [25]:
text

'\n            Новосибирский зоопарк регулярно рассказывает о детенышах, появившихся на свет в семьях животных. Ежегодно мы публикуем десятки историй о малышах. При этом многое остается нерассказанным, потому что коллекция очень большая. Но все события, все рождения фиксируются в документах, остаются в нашей летописи. Сегодня мы публикуем некоторые цифры, которые показывают результаты работы зоологического парка по получению потомства от животных и сохранению биологического разнообразия. \n\r\n В настоящее время в коллекции Новосибирского зоопарка представлено 803 вида, свыше 10 тысяч особей. В 2024 году было получено потомство от более 270 видов животных. Малыши рождались в семьях хищников, копытных, птиц, приматов, грызунов, рептилий, и все получали максимум заботы. \n\r\n У млекопитающих в общей сложности родилось более 250 детенышей. Среди них были малыши редких исчезающих видов, и каждый такой детеныш очень важен. Историческим событием стало рождение дальневосточного леопарда. Сре

Вот здесь можно [посмотреть часть документации BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html)

### Пишем краулер

Теперь давайте обкачаем несколько новостей.

1. Где взять ссылки на новости?

In [30]:
news_url = 'https://zoonovosib.ru/news/'
response = requests.get(news_url,
                        headers={'User-Agent':user_agent},
                        verify=False)
soup = BeautifulSoup(response.text, 'html.parser')

In [39]:
soup.find_all('a', {'class':"text-next"})[0]

<a class="text-next" href="/news/skolko-malyshey-rozhdaetsya-v-novosibirskom-zooparke-ezhegodno/">Читать подробнее <i class="gicon-arrow-right"></i></a>

2. Как вытащить ссылки из тегов?

In [29]:
for a in soup.find_all('a', {'class':"text-next"}):
    print(a['href'])

/news/skolko-malyshey-rozhdaetsya-v-novosibirskom-zooparke-ezhegodno/
/news/vesennyaya-dekada-pozhilogo-cheloveka-2025/
/news/lvitsa-naytiriya-otmechaet-den-rozhdeniya/
/news/medvedi-prosnulis-posle-zimney-spyachki/
/news/s-dnem-rozhdeniya-katanga-/
/news/pokazatelnye-kormleniya-15-16-marta/
/news/14-marta-mezhdunarodnyy-den-rek/
/news/3-marta-vsemirnyy-den-dikoy-prirody/
/news/s-1-marta-menyaetsya-rezhim-raboty-zooparka/
/news/pokazatelnye-kormleniya-1-2-marta/


3. Как ходить по страницам списков новостей, чтобы доставать ссылки?

In [40]:
news_urls = []
news_url = 'https://zoonovosib.ru/news/?PAGEN_1='
for i in range(1, 6):
    response = requests.get(news_url+str(i),
                        headers={'User-Agent':user_agent},
                        verify=False)
    soup = BeautifulSoup(response.text, 'html.parser')
    for a in soup.find_all('a', {'class':"text-next"}):
        news_urls.append('https://zoonovosib.ru/'+a['href'])
news_urls

['https://zoonovosib.ru//news/skolko-malyshey-rozhdaetsya-v-novosibirskom-zooparke-ezhegodno/',
 'https://zoonovosib.ru//news/vesennyaya-dekada-pozhilogo-cheloveka-2025/',
 'https://zoonovosib.ru//news/lvitsa-naytiriya-otmechaet-den-rozhdeniya/',
 'https://zoonovosib.ru//news/medvedi-prosnulis-posle-zimney-spyachki/',
 'https://zoonovosib.ru//news/s-dnem-rozhdeniya-katanga-/',
 'https://zoonovosib.ru//news/pokazatelnye-kormleniya-15-16-marta/',
 'https://zoonovosib.ru//news/14-marta-mezhdunarodnyy-den-rek/',
 'https://zoonovosib.ru//news/3-marta-vsemirnyy-den-dikoy-prirody/',
 'https://zoonovosib.ru//news/s-1-marta-menyaetsya-rezhim-raboty-zooparka/',
 'https://zoonovosib.ru//news/pokazatelnye-kormleniya-1-2-marta/',
 'https://zoonovosib.ru//news/pokazatelnye-kormleniya-22-23-fevralya/',
 'https://zoonovosib.ru//news/novosibirskiy-zoopark-mesto-samykh-neobychnykh-ekskursiy-/',
 'https://zoonovosib.ru//news/19-fevralya-den-ornitologa/',
 'https://zoonovosib.ru//news/amurskaya-tigritsa-u

In [41]:
# вытащим тексты этих новостей
news_texts = []
for link in news_urls:
    response = requests.get(link,
                        headers={'User-Agent':user_agent},
                        verify=False)
    soup = BeautifulSoup(response.text, 'html.parser')
    post = soup.find('div', {'class':"detail-text"}).get_text()
    news_texts.append(post.strip())

In [43]:
news_texts[5]

'Друзья, ждем вас на показательные кормления в эти выходные!\n\n15 марта, суббота\n\r\n 14:00 - Северные олени\r\n Уличные вольеры\r\n 15:00 - Ленивцы\r\n Павильон мелких приматов и пингвинов\r\n 15:30 - Снежный барс\r\n Уличные вольеры\r\n 16:00 - Кустарниковые собаки\r\n Павильон муравьедов и кустарниковых собак\n\n16 марта, воскресенье\n\r\n 14:00 - Земноводные, водяные агамы\r\n Террариум\r\n 15:00 - Полосатый варан\r\n Павильон мелких приматов и пингвинов\r\n 15:00 - Лемуры катта\r\n Павильон тропических животных\n\r\n Все выходные с 12:00 до 17:00 на главной площади около фонтана вы можете прокатиться на лошадке или пони. Услуга оплачивается дополнительно.\n\r\nЖдем вас в гости всей семьей или с друзьями в наш зоопарк!'

In [33]:
len(news_texts)

50